### 데이터 가져오기

In [1]:
from scrapy.http import TextResponse
from fake_useragent import UserAgent
import re

In [2]:
url = "http://emart.ssg.com/disp/category.ssg?dispCtgId=6000097635&page=2"

In [3]:
req = requests.get(url, headers={"User-Agent": UserAgent().chrome})

In [4]:
response = TextResponse(req.url, body=req.text, encoding="utf-8")

In [5]:
response

<200 http://emart.ssg.com/disp/category.ssg?dispCtgId=6000097635&page=2>

### 마지막 페이지 구하는 함수

In [6]:
last_page = response.xpath('//*[@id="area_itemlist"]/div[2]/a/@onclick').extract()[-1]
print(last_page)
last_p = int(re.findall('\d+', last_page)[0]) +1
last_p

itemLister.changePage(24);return false;


25

In [7]:
def last_page(category):
    url = "http://emart.ssg.com/disp/category.ssg?dispCtgId={}".format(category)
    req = requests.get(url, headers={"User-Agent": UserAgent().chrome})
    time.sleep(5)
    response = TextResponse(req.url, body=req.text, encoding="utf-8")

    # 페이지 수가 2개 이상인 카테고리
    try:
        last = response.xpath('//*[@id="area_itemlist"]/div[2]/a/@onclick').extract()[-1]
        last_page = int(re.findall('\d+', last)[0]) +1
        print("끝페이지 : ", last_page-1)
        
    # 페이지 수가 1개인 카테고리
    except:
        last_page = 2
        print("끝페이지: ", last_page-1)

    return last_page

In [8]:
# 함수 확인

last_page("6000097654")

끝페이지:  1


2

### title, price, link 가져오는 함수

In [9]:
def get_items(category, last_p):
    dfs = pd.DataFrame()

    for p in range(1, last_p): 
        url = "http://emart.ssg.com/disp/category.ssg?dispCtgId={}&page={}".format(category, p)

        req = requests.get(url, headers={"User-Agent": UserAgent().chrome})
        time.sleep(5)
        response = TextResponse(req.url, body=req.text, encoding="utf-8")

        elements = response.xpath('//*[@id="ty_thmb_view"]/ul/li')
        print("페이지 상품수 : ", len(elements), "\t", "페이지 번호", p)

        # title
        titles = response.xpath('//*[@id="ty_thmb_view"]/ul/li/div[@class="cunit_info"]/div[@class="cunit_md notranslate"]/div[@class="title"]/a/em[@class="tx_ko"]/text()').extract()

        # price
        price = response.xpath('//*[@id="ty_thmb_view"]/ul/li/div[@class="cunit_info"]/div[@class="cunit_price notranslate"]/div[@class="opt_price"]/em[@class="ssg_price"]/text()').extract()

        # link
        links = response.xpath('//*[@id="ty_thmb_view"]/ul/li/div[1]/div[2]/a/@href').extract()
        links = [response.urljoin(link) for link in links]


        df = pd.DataFrame({"titles": titles, "price": price, "links": links})
        dfs = pd.concat([dfs, df])

    dfs.reset_index(drop=True).tail()
    
    return dfs

In [10]:
dfs = get_items("6000095501", 2)

페이지 상품수 :  80 	 페이지 번호 1


In [11]:
# 컬럼별 아이템 수 확인

len(dfs["titles"]), len(dfs["price"]), len(dfs["links"])

(80, 80, 80)

### price per gram, minimum gram 뽑아내기

#### Price per gram => Xpath 이용

In [12]:
test_url = "http://emart.ssg.com/disp/category.ssg?dispCtgId=6000097653"
test_req = requests.get(test_url, headers={"User-Agent": UserAgent().chrome})
test_res = TextResponse(test_req.url, body=test_req.text, encoding="utf-8")

In [13]:
len(test_res.xpath('//*[@id="ty_thmb_view"]/ul/li'))

80

In [14]:
len(test_res.xpath('//*[@id="ty_thmb_view"]/ul/li/div[@class="cunit_info"]/div[@class ="cunit_prw"]/div/text()').extract())

21

In [15]:
idx = range(0,80)
pprs = []
for n in idx:
    ppr = test_res.xpath('//*[@id="ty_thmb_view"]/ul/li[{}]/div[@class="cunit_info"]/div[@class ="cunit_prw"]/div/text()'.format(n)).extract()
    ppr = ["-" if ppr == [] else ppr[0]]
    pprs.append(ppr[0])

In [16]:
len(pprs), pprs

(80,
 ['-',
  '-',
  '(10g당:242원)',
  '(10g당:173원)',
  '(10g당:179원)',
  '(10g당:223원)',
  '(10g당:235원)',
  '(10g당:180원)',
  '(10g당:264원)',
  '(10g당:232원)',
  '(10g당:250원)',
  '(10g당:165원)',
  '(10g당:250원)',
  '(10g당:142원)',
  '(10g당:315원)',
  '(10g당:221원)',
  '-',
  '-',
  '-',
  '-',
  '-',
  '(1000ea당:4,300원)',
  '-',
  '(100ea당:1,608원)',
  '(454g당:6,425원)',
  '-',
  '(1ea당:13,900원)',
  '-',
  '-',
  '-',
  '-',
  '-',
  '-',
  '-',
  '-',
  '-',
  '-',
  '-',
  '-',
  '-',
  '-',
  '-',
  '(100g당:3,143원)',
  '-',
  '(10g당:316원)',
  '(10g당:388원)',
  '-',
  '-',
  '-',
  '-',
  '-',
  '-',
  '-',
  '-',
  '-',
  '-',
  '-',
  '-',
  '-',
  '-',
  '-',
  '-',
  '-',
  '-',
  '-',
  '-',
  '-',
  '-',
  '-',
  '-',
  '-',
  '-',
  '-',
  '-',
  '-',
  '-',
  '-',
  '-',
  '-',
  '-'])

#### minimum gram =>  Regex 이용

In [17]:
# 유제품 카테고리

category = "6000095501"
last_p = 2
dfs = get_items(category, last_p)

페이지 상품수 :  80 	 페이지 번호 1


In [18]:
# ml, ML, l, L 뽑기
dfs["test"] = dfs["titles"].apply(lambda data: data.replace(" ", ""))
dfs["test"] = dfs["test"].apply(lambda data: re.findall('(\d+.?\d?[l|L]\*?x?X?\d?\d?)', data)[::-1][0] if re.findall('(\d+.?\d?[l|L]\*?x?X?\d?\d?)', data) else data)

In [19]:
dfs["titles"].head(30)

0                                     [서울우유] 서울우유 2.3L
1                                 [피코크] 우유(1등급) 1000ml
2                     [서울우유] 후레쉬 밀크 기획(900ml*2) 1800ml
3                             [서울우유] 비요뜨 초코링 (143g*2개)
4                             [서울우유] 서울우유 1000ml 나100%
5                          [노브랜드] 마시는 요구르트 65ml * 15개입
6                           [한국야쿠르트] 야쿠르트 라이트 65mlX15개
7                            [매일] 우유 오리지널 후레쉬팩 900ML*2
8                           [서울우유] 목장의 신선함이 살아있는 우유 1L
9                      [서울우유] 체다 슬라이스 치즈(300g2개입) 600g
10                          [서울우유] 커피 삼각 포리 (200ml*3개)
11        ※점포별 구성상이 [덴마크] 대니쉬 the건강한우유(900ml*2) 1800ml
12                              [남양] 고소한 저지방우유 900ml*2
13                                   [피코크] 그릭 요거트 450g
14                                [매일] 매일 바이오 플레인 450g
15                                    소와나무 체다치즈클래식204g
16                            [남양] 떠먹는 불가리스 기획 85g*10입
17                               [매일] 소화가 잘되는 우유 930ml
18        

In [20]:
dfs["test"].head(30)

0                             2.3L
1                           1000ml
2                           1800ml
3            [서울우유]비요뜨초코링(143g*2개)
4                           1000ml
5                          65ml*15
6                          65mlX15
7                          900ML*2
8                               1L
9      [서울우유]체다슬라이스치즈(300g2개입)600g
10                         200ml*3
11                          1800ml
12                         900ml*2
13                  [피코크]그릭요거트450g
14                [매일]매일바이오플레인450g
15                 소와나무체다치즈클래식204g
16            [남양]떠먹는불가리스기획85g*10입
17                           930ml
18                            1.8L
19               [서울우유]짜요짜요딸기맛240g
20                           960ml
21                         150mlX5
22                           500ml
23                          1680ml
24               [서울우유]짜요짜요포도맛240g
25          [남양]드빈치자연방목치즈255g*2입기획
26    [빙그레]요플레10개입기획85g*10(맛은랜덤배송)
27                 [피코크]플레인요거트900g
28                  

In [21]:
# 냉동과일 카테고리

category_2 = "6000096009"
last_p = 2
dfs_2 = get_items(category_2, last_p)

페이지 상품수 :  80 	 페이지 번호 1


In [22]:
# kg, KG, g, G 뽑기

dfs_2["test"] = dfs_2["titles"].apply(lambda data: data.replace(" ", ""))
dfs_2["test"] = dfs_2["test"].apply(lambda data: re.findall('(\d+.?\d?k?K?[g|G]\*?x?X?\d?\d?)', data)[::-1][0] if re.findall('(\d+.?\d?k?K?[g|G]\*?x?X?\d?\d?)', data) else data)

In [23]:
dfs_2["titles"].head(30)

0            [미국산] 냉동 블루베리 1.5kg/봉
1             [페루산] 냉동 애플망고 2.27kg
2           [미국산] 냉동 블루베리 1.13kg/팩
3              [페루산] 냉동 애플 망고 300g
4     [국내산] 두번 엄선한 국산 냉동딸기1kg x 2봉
5               [베트남산] 스위트망고 1.3kg
6                 냉동 유기농 블루베리 700g
7                 두번 엄선한 오색베리 500g
8                [칠레산] 냉동체리 700g/봉
9                   냉동블루베리+망고 기획세트
10            칠레산 냉동 트리플베리 1kg (팩)
11             칠레산 냉동 블루베리 1kg (팩)
12          [생활앤] 베트남산 냉동망고 1kg(팩)
13              페루산 냉동애플망고 1kg (팩)
14             미국산 냉동 블루베리 1kg (팩)
15                  조각 파인애플 300g/팩
16          [자연원] 국산 냉동 딸기 1kg *2봉
17          [코스타리카산] 냉동 파인애플 1.3kg
18                 냉동 딸기바나나 믹스 1KG
19                    유기농 냉동딸기500g
20          [자연원] 페루산 아보카도 500g(팩)
21          생활앤 페루산 냉동딸기 1.3kg (팩)
22          냉동 파인애플 1kg + 냉동망고 1kg
23           페루산 냉동 잉카블루베리 500g(팩)
24               껍질벗긴 아이스홍시 70g*6입
25          [자연원] 칠레산 냉동체리 700g(팩)
26               [국내산] 냉동오디 500g/봉
27       [자연원]페루산 냉동 왕 블루베리 1kg(팩)
28     냉동과일 미니팩 3종(블

In [24]:
dfs_2["test"].values

array(['1.5kg', '27kg', '13kg', '300g', '1kgx2', '1.3kg', '700g', '500g',
       '700g', '냉동블루베리+망고기획세트', '1kg', '1kg', '1kg', '1kg', '1kg', '300g',
       '1kg*2', '1.3kg', '1KG', '500g', '500g', '1.3kg', '1kg', '500g',
       '70g*6', '700g', '500g', '1kg', '냉동과일미니팩3종(블루베리,애플망고,딸기)', '1kg',
       '450g', '1.5kg', '1kg', '700g', '750g', '500g', '500g',
       '[자연원]베트남산프룻바망고10080*6입', '300g', '1kg', '80g', '400g', '220g',
       'PICKUP사과믹스', '1kg', '200g', '1kg', '500g', '13kgx3', '1kg',
       '1.3kgx2', '13kgx1', '1kgx2', '908gX1', '500gx1', '700gx3', '500g',
       '1kg', '1kg', '500gx3', '3kg', '1kg', '13kgx2', '400gx3', '1kgx2',
       '1KG', '1kgx2', '60gX25', '2kg', '1.3kgx1', '1kg', '1kg', '1kgx4',
       '[냉동과일1+1]패션후르츠/리치/망고스틴/블루베리등13종', '1kgx2', '1kgx3', '1kg', '1kg',
       '1kg', '1kgx4'], dtype=object)

#### [min_gram] 컬럼 만들기 => IF, ELIF, ELSE

In [25]:
def add_min_gram(data):
    
    data = data.replace(" ", "")

    if re.findall('(\d+.?\d?[l|L]\*?x?X?\d?\d?)', data):
        data = re.findall('(\d+.?\d?[l|L]\*?x?X?\d?\d?)', data)[::-1][0]

    elif re.findall('(\d+.?\d?k?K?[g|G]\*?x?X?\d?\d?)', data):
        data = re.findall('(\d+.?\d?k?K?[g|G]\*?x?X?\d?\d?)', data)[::-1][0]    
        
    else:
        data = "-"
        
    return data

In [26]:
dfs["test"] = dfs["titles"].apply(lambda data: add_min_gram(data))

In [27]:
dfs["test"].values

array(['2.3L', '1000ml', '1800ml', '143g*2', '1000ml', '65ml*15',
       '65mlX15', '900ML*2', '1L', '600g', '200ml*3', '1800ml', '900ml*2',
       '450g', '450g', '204g', '85g*10', '930ml', '1.8L', '240g', '960ml',
       '150mlX5', '500ml', '1680ml', '240g', '255g*2', '85g*10', '900g',
       '1000ml', '80g*3', '930mlx2', '900ml', '143g*2', '140ml20',
       '900ml', '200g', '900g', '1200ml', '136g*2', '80g4', '110mlX4',
       '125g', '950ml', '1300ml', '500g', '200ml*3', '1800ml', '85g4',
       '1000ml', '240g', '2.3L', '90g8', '190ml*20', '600ml', '200g',
       '900g', '400g', '200ml*3', '95g8', '150mlX5', '85g*4', '1200ml',
       '900ML*2', '144g', '200g', '450ml', '190ml16', '2.3L', '180g',
       '200ml*3', '100mlX5', '1440ml', '3130ml*4', '600ml', '450g', '1kg',
       '930ml', '930ml', '83g*10', '900g'], dtype=object)

### 카테고리 번호 => 이름 변환 함수

In [28]:
def category_name(category_num):
    names = {"냉동상품" : ["6000097651","6000097652","6000097653","6000097640","6000097639","6000097702","6000097703","6000097704","6000097705","6000097706","6000097707"],\
         "냉장식품" : ["6000097656","6000097658","6000097669","6000097671","6000097670"],\
         "식용유/설탕/소금": ['6000095598',"6000095711","6000095727","6000095709"],\
         "장류/조미료" : ["6000095605","6000095607","6000095608","6000095604","6000095724","6000095606","6000095712","6000095726","6000095728","6000095729","6000095713","6000095714","6000095715"],\
         "소스/양념/육수" : ["6000095591","6000095590","6000095592","6000095594","6000095609","6000095725","6000095612","6000095593"],\
         "밀가루/라면/믹스류": ["6000095595","6000097134"],\
         "편의식/가공기타" : ["6000097196","6000097637","6000097136","6000095508","6000097138","6000097137","6000095862","6000095506","6000096326"],\
         "반찬류" : ["6000097506", "6000097507"],\
         "과일" : ["6000095796", "6000095794"],\
         "포장용기" : ["0006510363","0006510361","0006510364","0006510365","0006510366","0006510546"],\
         "가게위생용품" : ["0006510362","0006510367"],\
         "유제품": "6000095501",\
         "야채/채소" : "6000095740",\
         "쌀/잡곡/견과" : "6000095498",\
         "수산/건어물" : "6000095500",\
         "축산/계란" : "6000095499"}
    for name, num in names.items():
        if num == category_num:
            return name       

In [29]:
# 함수 확인

category_name("6000095499")

'축산/계란'

### 크롤링 Class 생성

In [30]:
# categories = ["6000097651","6000097652","6000097653","6000097640","6000097639","6000097702","6000097703","6000097704","6000097705","6000097706","6000097707","6000097656","6000097658","6000097669","6000097671","6000097670",'6000095598',"6000095711","6000095727","6000095709", "6000095605","6000095607","6000095608","6000095604","6000095724","6000095606","6000095712","6000095726","6000095728","6000095729","6000095713","6000095714","6000095715", "6000095591","6000095590","6000095592","6000095594","6000095609","6000095725","6000095612","6000095593", "6000095595","6000097134", "6000097196","6000097637","6000097136","6000095508","6000097138","6000097137","6000095862","6000095506","6000096326", "6000097506", "6000097507", "6000095796", "6000095794", "0006510363","0006510361","0006510364","0006510365","0006510366","0006510546", "0006510362","0006510367","6000095501", "6000095740", "6000095498", "6000095500", "6000095499"]

In [31]:
%%writefile emart_local.py

from scrapy.http import TextResponse
from fake_useragent import UserAgent
import re
import time
import pandas as pd

from sqlalchemy import *
import MySQLdb
import configparser


class Emart:
    def __init__(self):
        self.test_cate = ["6000095695"]
        self.categories = ["6000097651","6000097652","6000097653","6000097640","6000097639","6000097702","6000097703","6000097704","6000097705","6000097706","6000097707","6000097656","6000097658","6000097669","6000097671","6000097670",'6000095598',"6000095711","6000095727","6000095709", "6000095605","6000095607","6000095608","6000095604","6000095724","6000095606","6000095712","6000095726","6000095728","6000095729","6000095713","6000095714","6000095715", "6000095591","6000095590","6000095592","6000095594","6000095609","6000095725","6000095612","6000095593", "6000095595","6000097134", "6000097196","6000097637","6000097136","6000095508","6000097138","6000097137","6000095862","6000095506","6000096326", "6000097506", "6000097507", "6000095796", "6000095794", "0006510363","0006510361","0006510364","0006510365","0006510366","0006510546", "0006510362","0006510367","6000095501", "6000095740", "6000095498", "6000095500", "6000095499"]
    
    
    
    def last_page(self, category):
        url = "http://emart.ssg.com/disp/category.ssg?dispCtgId={}".format(category)
        req = requests.get(url, headers={"User-Agent": UserAgent().chrome})
        time.sleep(5)
        response = TextResponse(req.url, body=req.text, encoding="utf-8")

        try:
            last = response.xpath('//*[@id="area_itemlist"]/div[2]/a/@onclick').extract()[-1]
            last_page = int(re.findall('\d+', last)[0]) + 1
            print("끝페이지 : ", last_page-1)

        except:
            last_page = 2
            print("끝페이지: ", last_page-1)

        return last_page

    
    
    def emart_crawling(self):
        dfs = pd.DataFrame()

        for category in self.categories:
            print("=== 카테고리 번호 : ", category, " ===")
            last_page = self.last_page(category)

            for p in range(1, last_page):
                url = "http://emart.ssg.com/disp/category.ssg?dispCtgId={}&page={}".format(category, p)

                req = requests.get(url, headers={"User-Agent": UserAgent().chrome})
                time.sleep(5)
                response = TextResponse(req.url, body=req.text, encoding="utf-8")

                elements = response.xpath('//*[@id="ty_thmb_view"]/ul/li')
                print("페이지 번호", p, "\t", "상품 수 : ", len(elements))

                # category
                cate = [category]*len(elements)

                # title
                titles = response.xpath('//*[@id="ty_thmb_view"]/ul/li/div[@class="cunit_info"]/div[@class="cunit_md notranslate"]/div[@class="title"]/a/em[@class="tx_ko"]/text()').extract()

                # price
                price = response.xpath('//*[@id="ty_thmb_view"]/ul/li/div[@class="cunit_info"]/div[@class="cunit_price notranslate"]/div[@class="opt_price"]/em[@class="ssg_price"]/text()').extract()

                # link
                links = response.xpath('//*[@id="ty_thmb_view"]/ul/li/div[1]/div[2]/a/@href').extract()
                links = [response.urljoin(link) for link in links]


                # price per gram
                idx = range(0,len(elements))
                pprs = []
                for n in idx:
                    ppr = response.xpath('//*[@id="ty_thmb_view"]/ul/li[{}]/div[@class="cunit_info"]/div[@class ="cunit_prw"]/div/text()'.format(n)).extract()
                    ppr = ["-" if ppr == [] else ppr[0]]
                    pprs.append(ppr[0])

                df = pd.DataFrame({"category": cate, "titles": titles, "price": price, "links": links, "price/gram": pprs})
                dfs = pd.concat([dfs, df])

        return dfs

        
        
    def min_gram(self, data):

        datas = data.replace(" ", "")

        if re.findall('(\d+.?\d?[l|L]\*?x?X?\d?\d?)', datas):
            datas = re.findall('(\d+.?\d?[l|L]\*?x?X?\d?\d?)', datas)[::-1][0]

        elif re.findall('(\d+.?\d?k?K?[g|G]\*?x?X?\d?\d?)', datas):
            datas = re.findall('(\d+.?\d?k?K?[g|G]\*?x?X?\d?\d?)', datas)[::-1][0]

        else:
            datas = "-"

        return datas
    
    

    def category_name(self, category):
        names = {"냉동상품": ["6000097651", "6000097652", "6000097653", "6000097640", "6000097639", "6000097702", "6000097703", "6000097704", "6000097705", "6000097706", "6000097707"],
                 "냉장식품": ["6000097656", "6000097658", "6000097669", "6000097671", "6000097670"],
                 "식용유/설탕/소금": ['6000095598', "6000095711", "6000095727", "6000095709"],
                 "장류/조미료": ["6000095605", "6000095607", "6000095608", "6000095604", "6000095724", "6000095606", "6000095712", "6000095726", "6000095728", "6000095729", "6000095713", "6000095714", "6000095715"],
                 "소스/양념/육수": ["6000095591", "6000095590", "6000095592", "6000095594", "6000095609", "6000095725", "6000095612", "6000095593"],
                 "밀가루/라면/믹스류": ["6000095595", "6000097134"],
                 "편의식/가공기타": ["6000097196", "6000097637", "6000097136", "6000095508", "6000097138", "6000097137", "6000095862", "6000095506", "6000096326"],
                 "반찬류": ["6000097506", "6000097507"],
                 "과일": ["6000095796", "6000095794"],
                 "포장용기": ["0006510363", "0006510361", "0006510364", "0006510365", "0006510366", "0006510546"],
                 "가게위생용품": ["0006510362", "0006510367"],
                 "유제품": "6000095501",
                 "야채/채소": "6000095740",
                 "쌀/잡곡/견과": "6000095498",
                 "수산/건어물": "6000095500",
                 "축산/계란": "6000095499"}
        for name, num in names.items():
            if num == category:
                return name

            
            
    def sql_db(self):
        config = configparser.ConfigParser()
        config.read(".local.ini")
        sql_datas = config["sql"]

        engine = create_engine("mysql://{}:{}@{}/crawling?charset=utf8".format(sql_datas["user"], sql_datas["pw"], sql_datas["public_ip"]), encoding='utf8')

        dfs = self.emart_crawling()
        dfs["min_gram"] = dfs["titles"].apply(lambda data: self.min_gram(data))
        dfs = dfs.reset_index(drop=True)
        dfs.to_sql(name="emart_goods", con=engine, if_exists="replace")
        print("saved in mySQL database!!")
        
        return dfs

Overwriting emart_local.py
